In [3]:
import numpy as np
from math import exp, sqrt
import networkx as nx
from scipy.integrate import solve_ivp
from Diff import *

In [2]:

sizes = [6, 5, 4]
probs = [[1, 0.05, 0.05], [0.05, 1, 0.05], [0.05, 0.05, 1]]
G = nx.stochastic_block_model(sizes, probs, seed=0)
pos = nx.spring_layout(G)
A = nx.adjacency_matrix(G).todense()
L = nx.laplacian_matrix(G).toarray()


n = G.number_of_nodes()
x0 = np.zeros(n)
activ = np.random.choice(n, int(0.2*n))
x0[activ] = 100

/var/folders/4g/4249y1xd42d75jmm55xxl99m0000gn/T/ipykernel_43106/3879556483.py:5: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G).todense()
<class 'networkx.utils.decorators.argmap'> compilation 16:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


In [5]:
c_bar = np.array([[20,3,5],
              [3,15,2],
              [5,2,10]])
C = construct_C(c_bar, sizes, n)
D = construct_D(A, C, n)
M = C*A - D
tau=0.1
T=10

In [32]:
A

matrix([[0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
        [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]])

In [6]:
lmb, v = np.linalg.eig(M)
data = np.array(x0)

In [29]:
a_0 = (np.linalg.inv(v)@x0)[0,:]


matrix([[-3.62484813e-01,  1.60221981e-01, -7.27436281e-02,
          3.24131385e-01, -8.37131760e-01, -2.88096478e-02,
          1.95535858e-02, -3.23010866e-02,  4.30057883e-18,
          1.26414146e-17, -8.84037416e-18,  3.75053631e-16,
         -4.80963553e-16, -1.03462751e-15,  2.00093128e-15],
        [-3.65080727e-01,  1.62936632e-01, -7.49791876e-02,
          1.19546574e-01,  2.51987502e-01, -3.10396502e-02,
          2.20313151e-02, -3.65822623e-02, -5.60895087e-17,
         -1.62440690e-16,  5.09424228e-16, -4.33550065e-16,
          8.63698820e-01, -2.19191889e-01,  3.63795590e-03],
        [-3.65080727e-01,  1.62936632e-01, -7.49791876e-02,
          1.19546574e-01,  2.51987502e-01, -3.10396502e-02,
          2.20313151e-02, -3.65822623e-02, -6.94261414e-17,
         -1.91432948e-17, -4.17294734e-16,  3.72240211e-16,
         -2.30937567e-01, -3.30555789e-01,  7.88754316e-01],
        [-3.65080727e-01,  1.62936632e-01, -7.49791876e-02,
          1.19546574e-01,  2.51987502

In [ ]:

time = [i*tau for i in range(1, int(T/tau))]

for t in time:
    E = np.array([exp(i*t) for i in lmb]) # vector of eigenvalues exponentials
    a = np.diag(E)@a_0  # vector of 'a' coefficients
    x = np.sum(np.diag(a)@v.T, axis=0) # psi(t)
    data = np.column_stack((data, x))